In [1]:
import pandas as pd
import numpy as np
import re

In [163]:
# Import card data sourced from scryfall.com
!wget -O oracle-cards.json https://data.scryfall.io/oracle-cards/oracle-cards-20240521210234.json
# Import card list sourced from 17lands.com
!wget -O cards_list.csv https://17lands-public.s3.amazonaws.com/analysis_data/cards/cards.csv

df = pd.read_json('oracle-cards.json')
cl = pd.read_csv('cards_list.csv')

# Map NaN's in 'arena_id' from scryfall data to correct values from 17lands data
df_ss = df[['name', 'arena_id']]
cl_merge = cl.merge(df_ss, how='left', left_on='name',right_on='name')
na_lookup = cl_merge.loc[cl_merge['arena_id'].isna()][['name','id']]
na_lookup = na_lookup.set_index('name')
na_map = {k:v.values[0] for k,v in na_lookup.iterrows()}
mapped_ids = df['name'].map(na_map)
df['arena_id'] = df['arena_id'].fillna(mapped_ids)


# Card text cleaning functions
def replace_self_reference(card_text, card_name):
    return card_text.replace(card_name, 'cardname')
def replace_newline(card):
    return card.replace('\n', ' ')
def is_multiface(card):
    if card in ['transform', 'adventure', 'modal_dfc', 'split']: return 1
    else: return 0

# Drop all cards not in Arena
df.dropna(subset=['arena_id'], inplace = True)
# Filter out all non-engligh printings
df = df.loc[df['lang'] == 'en']
# Filter out all token cards
df = df.loc[df['layout'] != 'token']
# Standardize text for cards that refer to themselves
df['oracle_text'] = df.apply(lambda row: replace_self_reference(str(row['oracle_text']), row['name']), axis=1)
# Add column to indicate multi-modal cards
df['is_multiface'] = df.apply(lambda row: is_multiface(row['layout']), axis=1)
df.reset_index(drop=True, inplace=True)

# Extract multi-faced card text
def expand_dual_faced_cards(df):
    # Prepare a list to collect new rows
    new_rows = []
    # Track indices to potentially drop later
    updated_indices = []
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        if isinstance(row['card_faces'], list):
            combined_text = []
            combined_power = []
            combined_toughness = []
            # Process each face of the card
            for face in row['card_faces']:
                # Create a new row dictionary starting from the current row's data
                new_row = row.to_dict()
                # Update this new row with data from the current face
                new_row.update(face)
                # Remove the 'card_faces' key as it's no longer needed in the new row
                new_row.pop('card_faces', None)
                # Add the new row to the list
                new_rows.append(new_row)
                cleaned_text = replace_self_reference(new_row['oracle_text'], new_row['name'])
                combined_text.append(cleaned_text)
                combined_power.append(new_row['power'])
                combined_toughness.append(new_row['toughness'])
            # Populate oracle text with both cards' text
            df.loc[index, 'oracle_text'] = combined_text[0] + ' // ' + combined_text[1]
            # Populate power and toughness with values from the front card face
            df.loc[index, 'power'] = combined_power[0]
            df.loc[index, 'toughness'] = combined_toughness[0]
            # Mark the index to drop the original row later
            updated_indices.append(index)
    return df

df = expand_dual_faced_cards(df)

# Filter out Alchemy cards
alchemy_idx = []
for idx, cardname in enumerate(df['name']):
    if cardname[:2] == 'A-':
        alchemy_idx.append(idx)
df.drop(alchemy_idx, inplace=True)

COLUMNS_TO_KEEP = ['arena_id','name', 'layout','mana_cost', 'cmc', 'type_line','oracle_text', 'power',
                    'toughness', 'color_identity','keywords', 'set', 'rarity',
                    'is_multiface']

df['arena_id'] = df['arena_id'].astype(int)
df = df[COLUMNS_TO_KEEP]
df.reset_index(drop=True, inplace=True)


def get_secondary_types(type_line):
    if len(type_line.split(' — ')) <= 1:
        return np.nan
    else:
        return type_line.split(' — ')[1]


df['type_line'] = df.apply(lambda row: row['type_line'].split(' // ')[0], axis=1)
df['primary_type'] = df.apply(lambda row: row['type_line'].split(' — ')[0], axis=1)
df['secondary_type'] = df.apply(lambda row: get_secondary_types(row['type_line']), axis=1)
df['color_identity'] = df.apply(lambda row: ''.join(row['color_identity']), axis=1)

# Indicator variable for primary types
df['is_legend']   = df['primary_type'].apply(lambda x: 'Legendary' in x)
df['is_creature'] = df['primary_type'].apply(lambda x: 'Creature' in x)
df['is_instant']  = df['primary_type'].apply(lambda x: 'Instant' in x)
df['is_sorcery']  = df['primary_type'].apply(lambda x: 'Sorcery' in x)
df['is_enchant']  = df['primary_type'].apply(lambda x: 'Enchantment' in x)
df['is_artifact'] = df['primary_type'].apply(lambda x: 'Artifact' in x)
df['is_planeswalker'] = df['primary_type'].apply(lambda x: 'Planeswalker' in x)
df['is_land'] =  df['primary_type'].apply(lambda x: 'Land' in x)
# df['is_snow'] =  df['primary_type'].apply(lambda x: 'Snow' in x)
# df['is_battle'] = df['primary_type'].apply(lambda x: 'Battle' in x)
# df['is_horror'] = df['primary_type'].apply(lambda x: 'Horror' in x)
# df['is_tribal'] = df['primary_type'].apply(lambda x: 'Tribal' in x)
# df['is_basic'] = df['primary_type'].apply(lambda x: 'Basic' in x)

# Indicator variables for color identities
df['is_w'] = df['color_identity'].apply(lambda x: 'W' in x)
df['is_u'] = df['color_identity'].apply(lambda x: 'U' in x)
df['is_b'] = df['color_identity'].apply(lambda x: 'B' in x)
df['is_r'] = df['color_identity'].apply(lambda x: 'R' in x)
df['is_g'] = df['color_identity'].apply(lambda x: 'G' in x)

# Extract all keywords
keywords = []
def get_keywords(card):
    for x in card:
        keywords.append(x)
for card in df['keywords']:
    get_keywords(card)
keywords_list = list(set(keywords))

# Prepare one-hot dataframe for keywords
keyword_dict = {}
for keyword in keywords_list:
    keyword_dict[keyword] = [keyword in x for x in df['keywords']]
df_keywords = pd.DataFrame(keyword_dict)

# Set variables for evergreen keywords only
evergreen_keywords = ['Deathtouch','Defender','Double strike', 'Enchant', 'Equip', 'First strike',
                      'Flash', 'Flying', 'Haste', 'Hexproof', 'Indestructible', 'Lifelink', 'Menace',
                      'Protection', 'Reach', 'Trample', 'Vigilance']

df_evergreen = df_keywords[evergreen_keywords]
df_set_keywords = df_keywords.drop(columns=evergreen_keywords, axis=1)
df_set_keywords['set_keyword'] = (df_set_keywords != 0).any(axis=1)
df_keywords = df_evergreen.merge(df_set_keywords['set_keyword'], left_index=True, right_index=True)
df = df.join(df_keywords)


# Set a variable to iindicate double pips in mana cost
def has_double_pip(cost_string):
    # regex for any repeated colors in mana cost, like {W}{W}
    matches = re.findall(r'\{([^}]+)\}', cost_string)
    return any(matches.count(x) > 1 for x in set(matches))

df['mana_cost'] = df['mana_cost'].astype('str')
df['double_pip'] = df['mana_cost'].apply(has_double_pip)

# Set a variable to indicate a special layout (e.g. adventure, saga, etc.)
df['special_layout'] = df['layout'] != 'normal'

# Create new dataframe of relevant columns
tab_data = df.drop(['mana_cost', 'type_line', 'oracle_text','color_identity', 'set', 'keywords','primary_type','secondary_type', 'layout'], axis=1)

# Set power/toughness of non-creature cards to -2 for one-hot encoding
tab_data['power'] = tab_data['toughness'].fillna(-2)
tab_data['toughness'] = tab_data['toughness'].fillna(-2)
# Set special/variable power/toughness to -1 for one-hot encoding
tab_data['power'] = pd.to_numeric(tab_data['power'], errors='coerce').fillna(-1).astype(int)
tab_data['toughness'] = pd.to_numeric(tab_data['toughness'], errors='coerce').fillna(-1).astype(int)
# Clip power/toughness/cmc to 5
tab_data['cmc'] = tab_data['cmc'].clip(upper=5)
tab_data['power'] = tab_data['power'].clip(upper=5)
tab_data['toughness'] = tab_data['toughness'].clip(upper=5)
# One-hot encode cmc, power, toughness, and rarity
tab_data = pd.get_dummies(tab_data, columns=['cmc', 'power', 'toughness', 'rarity'])

# Reindex using arena_id
tab_data = tab_data.set_index('arena_id')
# Convert to numeric
# tab_data = tab_data.astype(float)
# Save to file
tab_data.to_parquet('card_features_reduced.parquet')
# Save oracle text to file for later use
df[['name','arena_id','oracle_text']].to_parquet('oracle_text.parquet')

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
